In [2]:
import pandas as pd
from transformers import pipeline

# CSV 파일 로딩
df = pd.read_csv("mergedDftest.csv", encoding="cp949")
df = df.dropna(subset=['리뷰 내용'])

# 감성 분석 모델 불러오기
sentiment_pipeline = pipeline("sentiment-analysis", model="snunlp/KR-FinBert-SC")

# 감성 분석 실행
review_texts = df['리뷰 내용'].tolist()
results = [sentiment_pipeline(text)[0] for text in review_texts]

# 결과 추가
df.loc[df.index[:len(results)], '감성 결과'] = [r['label'] for r in results]
df.loc[df.index[:len(results)], '감성 점수'] = [r['score'] for r in results]
df.loc[df.index[:len(results)], '감성 점수 정제'] = df.loc[df.index[:len(results)], '감성 결과'].map(
    lambda x: 1.0 if 'positive' in x.lower() else 0.5 if 'neutral' in x.lower() else 0.0
)


# 상품별 평균 감성 점수 계산
sentiment_score_by_item = df.loc[df.index[:len(results)]].groupby('상품 이름')['감성 점수 정제'].mean().to_dict()


Device set to use cpu


In [3]:
print(df['감성 결과'].value_counts())

감성 결과
neutral     3733
positive       8
negative       3
Name: count, dtype: int64


In [4]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import GridSearchCV

# CSV 불러오기
# df = pd.read_csv("mergedDf.csv", encoding="cp949")
df_sentiment = df.copy()
df = df[['리뷰 ID', '상품 이름', '별점']].dropna()
df['별점'] = pd.to_numeric(df['별점'], errors='coerce')
df = df.dropna(subset=['별점'])

# 사용자 필터링 (리뷰 수 3개 이상)
user_counts = df['리뷰 ID'].value_counts()
valid_users = user_counts[user_counts >= 3].index
df = df[df['리뷰 ID'].isin(valid_users)]

# Surprise용 데이터셋 준비
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['리뷰 ID', '상품 이름', '별점']], reader)

# Grid Search 파라미터
param_grid = {
    'n_factors': [50, 100, 150],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1]
}

# GridSearchCV 실행
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)
gs.fit(data)

# 최적 모델 정보 출력
print("✅ Best RMSE:", gs.best_score['rmse'])
print("✅ Best Params:", gs.best_params['rmse'])

# 최적 모델 재학습
best_model = gs.best_estimator['rmse']
trainset = data.build_full_trainset()
best_model.fit(trainset)


✅ Best RMSE: 0.39597233384534586
✅ Best Params: {'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.1}


In [5]:
from surprise import SVD, NMF, KNNBasic, KNNWithMeans, BaselineOnly, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# 추천용 데이터 준비
# df = pd.read_csv('감성분석결과.csv')
df_rating = df_sentiment[['리뷰 ID', '상품 이름', '별점']].dropna()
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_rating, reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# 여러 알고리즘 비교
algos = {
    "SVD": SVD(),
    "NMF": NMF(),
    "KNNBasic": KNNBasic(),
    "KNNWithMeans": KNNWithMeans(),
    "BaselineOnly": BaselineOnly()
}
best_algo = None
best_rmse = float('inf')

for name, algo in algos.items():
    model = algo
    model.fit(trainset)
    preds = model.test(testset)
    error = rmse(preds, verbose=False)
    print(f"{name} RMSE: {round(error, 4)}")
    if error < best_rmse:
        best_rmse = error
        best_algo = model

SVD RMSE: 0.4816
NMF RMSE: 0.6373
Computing the msd similarity matrix...
Done computing similarity matrix.
KNNBasic RMSE: 0.4905
Computing the msd similarity matrix...
Done computing similarity matrix.
KNNWithMeans RMSE: 0.5055
Estimating biases using als...
BaselineOnly RMSE: 0.4763


In [10]:
def hybrid_recommend(user_id, top_n=5, alpha=0.7):
    user_items = df[df['리뷰 ID'] == user_id]['상품 이름'].unique()
    all_items = df['상품 이름'].unique()
    unseen_items = [item for item in all_items if item not in user_items]

    recommendations = []
    if user_id not in trainset._raw2inner_id_users:
        print("⚠️ Cold-start 유저입니다. 감성 기반 추천만 사용합니다.")
        for item in unseen_items:
            senti_score = sentiment_score_by_item.get(item, 0.5)
            hybrid_score = senti_score * 5
            recommendations.append((item, hybrid_score))
    for item in unseen_items:
        cf_score = best_model.predict(user_id, item).est
        senti_score = sentiment_score_by_item.get(item, 0.5)
        hybrid_score = alpha * cf_score + (1 - alpha) * senti_score * 5
        recommendations.append((item, hybrid_score))

    return sorted(recommendations, key=lambda x: x[1], reverse=True)[:top_n]


# 예시 실행
print(hybrid_recommend("kenn ", top_n=5))


[('[사리원 SET] 왕만두 전골+소불고기', 4.25), ('[분식이지] 바지락칼국수와 바삭김치전', 4.25), ('쟌슨빌 부대찌개', 4.25), ('[사리원] 소불고기 전골', 4.25), ('간장 돼지 불고기(1인)', 4.25)]


In [7]:
df['리뷰 ID'].value_counts()

리뷰 ID
kenn     192
jhan      80
cjdt      73
cjh4      65
fams      65
        ... 
retr       3
cwki       3
hfs8       3
jyby       3
vict       3
Name: count, Length: 309, dtype: int64

In [8]:
import pandas as pd
import mysql.connector

# MySQL 연결 설정
conn = mysql.connector.connect(
    host = "pandas13.cafe24.com",
    user = "pandas13",
    password = "cookit@012",
    database = "pandas13"
) 

query = 'select * from comment'
df_member = pd.read_sql(query, conn)
df_member

C:\Users\admin\AppData\Local\Temp\ipykernel_12228\3138721.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_member = pd.read_sql(query, conn)


,product_name,product_image,user_id,user_rating,text,price,total_rating,category,commentcol
0,[VIPS] 빕스 클래식 스테이크,1_image.jpg,dudr,5,채소랑 함께 곁들여서 먹으니 맜있어요,20900,5,양식,
1,[VIPS] 빕스 클래식 스테이크,1_image.jpg,hwaf,5,너무 맛있게 먹었어요 ^^,20900,5,양식,
2,[VIPS] 빕스 클래식 스테이크,1_image.jpg,haer,5,가니쉬 까지 있어서 맛있었어요! 고기에 떡심이 있어서 질겼는데 앞으로는 좋은 부위 ...,20900,5,양식,
3,[VIPS] 빕스 클래식 스테이크,1_image.jpg,benn,5,배송도 빠르고 맛도 좋아요,20900,5,양식,
4,[VIPS] 빕스 클래식 스테이크,1_image.jpg,haer,5,시즈닝이랑 가니쉬가 너무너무 맛있음\n고기 가운데 심이 있는게 와서 그것만 빼면 완벽,20900,5,양식,
...,...,...,...,...,...,...,...,...,...
3746,[놀부] 놀부 김치 부대찌개,98_image.jpg,7wes,5,처음 먹어봤는데 맛있게 잘먹었어요. 포장되어있었을땐 조금인것같았으나 냄비에 넣고보니...,15900,5,한식,
3747,[놀부] 놀부 김치 부대찌개,98_image.jpg,leey,5,입맛없을때 식욕돋는 놀부김치부대찌개예요 3명이서 먹는다고 햄과 김치 더넣고 먹었어요...,15900,5,한식,
3748,[놀부] 놀부 김치 부대찌개,98_image.jpg,haneul,5,개꿀맛,15900,5,한식,
3749,버섯가득 소불고기전골,92_image.jpg,haneul,5,맛있어요,16900,5,한식,


In [9]:
self.df['user_rating'].value_counts()


NameError: name 'self' is not defined